# Ray Crash Course  - Distributed HPO with Ray Tune's TuneGridSearchCV and Scikit-Learn

© 2019-2022, Anyscale. All Rights Reserved

This demo introduces **Ray tune's** key concepts using a classification example. Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune. We'll use a drop-in replacement for normal Scikit-learn's `GridSearchCV` with distributed Ray Tune's `TuneGridSearchCV`.

See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [22]:
# Import Tune's replacement
from ray.tune.sklearn import TuneGridSearchCV

# Other relevant imports
from sklearn.model_selection import train_test_split

# Use the stochastic gradient descent (SGD) classifier
from sklearn.linear_model import SGDClassifier

# import the classification dataset
from sklearn.datasets import make_classification
import numpy as np
import time
import logging
import ray

In [23]:
CONNECT_TO_ANYSCALE=False
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-ray-core-tutorial")
    else:
        ray.init()

2022-04-19 11:19:38,906	INFO services.py:1460 -- View the Ray dashboard at http://127.0.0.1:8265


### Create Feature Set

 * 250K rows
 * 250 features
 * 2 classes

In [24]:
def create_classification_data() -> (np.ndarray, np.ndarray):
    X, y = make_classification(
        n_samples=250000,
        n_features=250,
        n_informative=50,
        n_redundant=0,
        n_classes=2,
        class_sep=2.5)
    return X, y

### Create classification data and define parameter search space

In [25]:
X, y = create_classification_data()
# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=10000)

# Example parameters grid to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1, 1], "epsilon": [0.01, 0.1]}

## Use Ray's Scikit-learn drop-in replacement TuneGridSearchCV
Use all cores on a Ray Cluster or local host to tune 

In [26]:
# Now let's do with Tune's in-place replacement
# Note: If early_stopping=True, TuneGridSearchCV will default to using Tune’s ASHAScheduler.
tune_sklearn = TuneGridSearchCV(SGDClassifier(), 
                    parameter_grid,
                    early_stopping=True,
                    max_iters=30,
                    n_jobs=12,    # Use 40 cores if running on a cluster
                    mode="min",
                    verbose=True)

In [27]:
%%time
tune_sklearn.fit(x_train, y_train)

2022-04-19 11:21:00,108	INFO tune.py:702 -- Total run time: 60.15 seconds (60.03 seconds for the tuning loop).


CPU times: user 2.03 s, sys: 1.28 s, total: 3.31 s
Wall time: 1min 1s


TuneGridSearchCV(early_stopping=True, estimator=SGDClassifier(), max_iters=30,
                 mode='min', n_jobs=12,
                 param_grid={'alpha': [0.0001, 0.1, 1], 'epsilon': [0.01, 0.1]},
                 sk_n_jobs=1, verbose=True)

In [29]:
print(f"Ray Tune Scikit-learn TuneGridSearchCV Best params: {tune_sklearn.best_params}")

Ray Tune Scikit-learn TuneGridSearchCV Best params: {'alpha': 0.1, 'epsilon': 0.1}


In [30]:
ray.shutdown()

### Homework

1. Try some [Tune How-to guides](https://docs.ray.io/en/latest/tune/examples/index.html) 